In [ ]:
import numpy as np
import sys, os
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', 500)
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
from scipy import signal 
from itertools import combinations
from sklearn.linear_model import LinearRegression

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

# Load data

In [ ]:
full_folder = Path(r'C:\Users\elizabeth\OneDrive - Biolinq\Shared Documents\Data\Aggregate\eblinqiblinq 14 - 19b\all_data\prospective_8S17_v14d_2022_5day')
loaded_data = FileRead.load_pickle('alg_out_no_excel.zip', full_folder, as_dict=True)
all_sensor_data = loaded_data['all_sensor_data']
sensor_map_df = loaded_data['sensor_map_df'].copy()

In [ ]:
mfg_df=pd.read_excel(r'C:\Users\elizabeth\OneDrive - Biolinq\Shared Documents\Data\Aggregate\eblinqiblinq 14 - 19b\manufacturing data\BLINQ MPI-29 TRENDING SENSOR DATA.xlsx',sheet_name='compiled sensor data',skiprows=7)
mfg_df

In [ ]:
bench_sims=pd.read_excel(r'C:\Users\elizabeth\OneDrive - Biolinq\Shared Documents\Data\Aggregate\eblinqiblinq 14 - 19b\manufacturing data\benchsims.xlsx',usecols='B:K')
bench_sims

# Create Features and aggregate

Parameters for feature generation

In [ ]:
ch_str = ['1','3','4']
combination_paired_ch = list(combinations(ch_str,2))
current_ch_str = '1'
col_prefix = 'v14_s'
col_suffix = '_model_output'
cols_to_use = []
for ch_str_iter in ch_str: cols_to_use.append(col_prefix+ch_str_iter+col_suffix)

roll_win_size = 96
roll_min_size_ratio = 0.5

feature_name_base = {
    'filtered': 'sg_filt_s',
    'gluc_noise': 'roll_snr_s',
    'gluc_var1_s': 'roll_std_s',
    'gluc_var2_s': 'roll_diff_maxmin_s',
    'gluc_var3_s': 'roll_iqr_s',
    'gluc_mov_avg1_s': 'roll_mean_s',
    'gluc_mov_avg2_s': 'roll_median_s',
    'gluc_avg_s': 'expand_median_s',
    'gluc_change1': 'magnitude_change_s',
    'gluc_change2': 'trend_change_s',
    'ch_bias': 'roll_mrd_8hr',
    'ch_precision': 'roll_mard_8hr',
    'ch_corr': 'roll_corr_8hr',
}

Start for loop

In [ ]:
aggregated_df=pd.DataFrame()
# for sensor in all_sensor_data.keys():
for sensor in ['31360030051', '31360030069', '31360030071', '31360020004', '31360020006', '31360020023', '31360020026', '31360020032', '31360020034', '31360020038', '31360020041']:
    sensor_df=all_sensor_data[sensor]['biolinq']

    ###calculate features###
    """ Calculate the features Liang defined """
    sensor_df=sensor_df.rename(columns={'Time':'Biolinq_Time'})
    for paired_ch in combination_paired_ch:
        col_A = col_prefix+paired_ch[0]+col_suffix
        col_B = col_prefix+paired_ch[1]+col_suffix
        diff_cols = sensor_df[col_A] - sensor_df[col_B]
        avg_cols = sensor_df[col_A].divide(2) + sensor_df[col_B].divide(2)
        rd_cols = diff_cols.divide(avg_cols).multiply(100)
        ard_cols = rd_cols.abs()
        sensor_df[feature_name_base['ch_bias']+''.join(paired_ch)] = rd_cols.rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).mean()
        sensor_df[feature_name_base['ch_precision']+''.join(paired_ch)] = ard_cols.rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).mean()
        sensor_df[feature_name_base['ch_corr']+''.join(paired_ch)] = sensor_df[col_A].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).corr(sensor_df[col_B])

    for current_ch_str in ch_str:
        current_col = col_prefix+current_ch_str+col_suffix
        not_current_ch_str = ch_str.copy()
        not_current_ch_str.remove(current_ch_str)
        not_current_col = []
        for ch_str_iter in not_current_ch_str: not_current_col.append(col_prefix+ch_str_iter+col_suffix) 
            
        current_model_output_filtered = pd.Series(signal.savgol_filter(sensor_df[current_col],24,2))
        sensor_df[feature_name_base['filtered']+current_ch_str] = current_model_output_filtered
        current_model_output_noise = sensor_df[current_col] - current_model_output_filtered
        sensor_df[feature_name_base['gluc_noise']+current_ch_str] = sensor_df[current_col].pow(2).rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).mean().\
                            divide(current_model_output_noise.pow(2).rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).mean())

        sensor_df[feature_name_base['gluc_var1_s']+current_ch_str] = sensor_df[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).std()
        sensor_df[feature_name_base['gluc_var2_s']+current_ch_str] = sensor_df[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).max()-\
                                        sensor_df[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).min()
        sensor_df[feature_name_base['gluc_var3_s']+current_ch_str] = sensor_df[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).quantile(0.75)-\
                                        sensor_df[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).quantile(0.25)

        sensor_df[feature_name_base['gluc_mov_avg1_s']+current_ch_str] = sensor_df[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).mean()
        sensor_df[feature_name_base['gluc_mov_avg2_s']+current_ch_str] = sensor_df[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).median()
        sensor_df[feature_name_base['gluc_avg_s']+current_ch_str] = sensor_df[current_col].expanding().median()

        sensor_df[feature_name_base['gluc_change1']+current_ch_str] = sensor_df[feature_name_base['gluc_mov_avg2_s']+current_ch_str].subtract(sensor_df[feature_name_base['gluc_avg_s']+current_ch_str])
        sensor_df[feature_name_base['gluc_change2']+current_ch_str] = sensor_df[feature_name_base['gluc_change1']+current_ch_str].diff().rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).median()
         
    #create matched pairs and aggregate
    sensor_agg=pd.DataFrame()
    for ref in ['ysi','acck','Dexcom','Libre']:
        if ref not in all_sensor_data[sensor].keys():
            continue
        ref_df=all_sensor_data[sensor][ref]

        ###adjust ref###

        ref_df=ref_df.rename(columns={ref:'Ref'})
        ref_df=ref_df.rename(columns={'Time':'Ref_Time'})
        ref_df['RefType']=ref
        
        
        paired_df=pd.merge_asof(right=sensor_df.dropna(subset=['UTC_Time']).sort_values(by='UTC_Time'),left=ref_df.dropna(subset=['UTC_Time']).sort_values(by='UTC_Time'),on='UTC_Time',direction='nearest',tolerance=pd.Timedelta('5min'))
        # paired_df=paired_df.dropna(subset=['Ref'])
        # paired_df=paired_df.dropna(subset=['biolinq'])
        
        sensor_agg=sensor_agg.append(paired_df)
    
    aggregated_df=aggregated_df.append(sensor_agg)

###calculate metrics###
aggregated_df['ARD']=100*np.abs((aggregated_df['biolinq']-aggregated_df['Ref'])/aggregated_df['Ref'])
aggregated_df['RD']=100*((aggregated_df['biolinq']-aggregated_df['Ref'])/aggregated_df['Ref'])
aggregated_df['D']=(aggregated_df['biolinq']-aggregated_df['Ref'])



# Load Data for Mac

In [ ]:
data_folder = Path(r'/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Documents - Clinical Data Analysis/Data/Aggregate/eblinqiblinq 14 - 19b/all_data/prospective_8S17_v14d_2022_5day')
loaded_data = FileRead.load_pickle('alg_out_no_excel.zip', data_folder, as_dict=True)
all_sensor_data = loaded_data['all_sensor_data']
sensor_map_df = loaded_data['sensor_map_df'].copy()
mfg_df=pd.read_excel(r'/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Documents - Clinical Data Analysis/Data/Aggregate/eblinqiblinq 14 - 19b/manufacturing data/BLINQ MPI-29 TRENDING SENSOR DATA.xlsx',sheet_name='compiled sensor data',skiprows=7)
bench_sims=pd.read_excel(r'/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Documents - Clinical Data Analysis/Data/Aggregate/eblinqiblinq 14 - 19b/manufacturing data/benchsims.xlsx',usecols='B:K')


# Test scripts

In [ ]:
print(loaded_data.keys())
all_sensor_id = list(loaded_data['all_sensor_data'].keys())
print(all_sensor_id)

In [ ]:
sensor_id = all_sensor_id[1]
print(sensor_id)
sensor_data = loaded_data['all_sensor_data'][sensor_id]
sensor_data.keys()

In [ ]:
biolinq_data = sensor_data['biolinq']
ysi_data = sensor_data['ysi']
acck_data = sensor_data['acck']
Dexcom_data = sensor_data['Dexcom']
Libre_data = sensor_data['Libre']
biolinq_data.head(100)
# ysi_data.head()
# acck_data.head()
# Dexcom_data.head()
# Libre_data.head()

In [ ]:
fig = make_subplots(rows=4, cols=2,shared_xaxes=True,vertical_spacing = 0.01,subplot_titles=['Sensor: ' + sensor_id])
fig = fig.update_layout(
    plot_bgcolor='white',
    height=1200,
    legend_tracegroupgap=10
)
fig = fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig = fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
current_row = 1
ysi_size= 10
acck_size = 8

In [ ]:
ch_str = ['1','3','4']
current_ch_str = '1'
col_prefix = 'v14_s'
col_suffix = '_model_output'

roll_win_size = 96
roll_min_size_ratio = 0.5

In [ ]:
cols_to_use = []
for ch_str_iter in ch_str: cols_to_use.append(col_prefix+ch_str_iter+col_suffix) 
current_col = col_prefix+current_ch_str+col_suffix
not_current_ch_str = ch_str.copy()
not_current_ch_str.remove(current_ch_str)
not_current_col = []
for ch_str_iter in not_current_ch_str: not_current_col.append(col_prefix+ch_str_iter+col_suffix) 

current_model_output_filtered = pd.Series(signal.savgol_filter(biolinq_data[current_col],24,2))
fig = fig.add_trace(go.Scatter(x=ysi_data['Time'],y=ysi_data['ysi'],name='YSI',legendgroup = '1',mode='markers',marker=dict(size=ysi_size,color='magenta',symbol='x')),row=current_row,col=1)
fig = fig.add_trace(go.Scatter(x=acck_data['Time'],y=acck_data['acck'],name='Acck',legendgroup = '1',mode='markers',marker=dict(size=acck_size,color='red',symbol='circle',line=dict(color='black',width=1))),row=current_row,col=1)
for other_col in not_current_col:
    fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[other_col],name=other_col,legendgroup = '1',line=dict(width=1,color='lightgray')),row=current_row,col=1)
fig = fig.add_trace(go.Scatter(x=Libre_data['Time'],y=Libre_data['Libre'],name='Libre',legendgroup = '1',mode='lines',line=dict(width=2,color='lime')),row=current_row,col=1)
fig = fig.add_trace(go.Scatter(x=Dexcom_data['Time'],y=Dexcom_data['Dexcom'],name='Dexcom',legendgroup = '1',mode='lines',line=dict(width=2,color='lightgreen')),row=current_row,col=1)
fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[current_col],mode='markers',name=current_col,legendgroup = '1',marker=dict(size=3,color='darkblue')),row=current_row,col=1)
fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=current_model_output_filtered,name=current_col+'_filtered',legendgroup = '1',line=dict(width=2.5,color='lightblue')),row=current_row,col=1)

In [ ]:
combination_paired_ch = list(combinations(ch_str,2))
for paired_ch in combination_paired_ch:
    if current_ch_str in paired_ch:
        col_A = col_prefix+paired_ch[0]+col_suffix
        col_B = col_prefix+paired_ch[1]+col_suffix
        # feature_name1 = 'Ard_'+''.join(paired_ch)
        feature_name2 = 'roll_mrd_8hr'+''.join(paired_ch)
        feature_name3 = 'roll_mard_8hr'+''.join(paired_ch)
        diff_cols = biolinq_data[col_A] - biolinq_data[col_B]
        avg_cols = biolinq_data[col_A].divide(2) + biolinq_data[col_B].divide(2)
        rd_cols = diff_cols.divide(avg_cols).multiply(100)
        ard_cols = rd_cols.abs()
        # biolinq_data[feature_name1] = diff_cols.abs().divide(avg_cols).multiply(100)
        biolinq_data[feature_name2] = rd_cols.rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).mean()
        biolinq_data[feature_name3] = ard_cols.rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).mean()

        feature_name4 = 'roll_corr_8hr_'+''.join(paired_ch)
        biolinq_data[feature_name4] = biolinq_data[col_A].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).corr(biolinq_data[col_B])

        # fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name1],name=feature_name1,legendgroup = '2'),row=2,col=1)
        fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name2],name=feature_name2,legendgroup = '2'),row=2,col=1)
        fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name3],name=feature_name3,legendgroup = '2'),row=2,col=1)
        fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name4],name=feature_name4,legendgroup = '3'),row=3,col=1)

current_model_output_noise = biolinq_data[current_col] - current_model_output_filtered
feature_name5 = 'roll_snr_s' + current_ch_str
biolinq_data[feature_name5] = biolinq_data[current_col].pow(2).rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).mean().\
                            divide(current_model_output_noise.pow(2).rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).mean())
fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name5],name=feature_name5,legendgroup = '4'),row=4,col=1)

feature_name6 = 'roll_std_s' + current_ch_str
feature_name7 = 'roll_diff_max_min_s' + current_ch_str
feature_name8 = 'roll_iqr_s' + current_ch_str
biolinq_data[feature_name6] = biolinq_data[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).std()
biolinq_data[feature_name7] = biolinq_data[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).max()-\
                                biolinq_data[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).min()
biolinq_data[feature_name8] = biolinq_data[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).quantile(0.75)-\
                                biolinq_data[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).quantile(0.25)

fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name6],name=feature_name6,legendgroup = '5'),row=1,col=2)
fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name7],name=feature_name7,legendgroup = '5'),row=1,col=2)
fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name8],name=feature_name8,legendgroup = '5'),row=1,col=2)

feature_name9 = 'roll_mean_s' + current_ch_str
feature_name10 = 'roll_median_s' + current_ch_str
feature_name11 = 'expand_median_s' + current_ch_str
biolinq_data[feature_name9] = biolinq_data[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).mean()
biolinq_data[feature_name10] = biolinq_data[current_col].rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).median()
biolinq_data[feature_name11] = biolinq_data[current_col].expanding().median()
fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name9],name=feature_name9,legendgroup = '6'),row=2,col=2)
fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name10],name=feature_name10,legendgroup = '6'),row=2,col=2)
fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name11],name=feature_name11,legendgroup = '6'),row=2,col=2)

feature_name12 = 'magnitude_change_' + current_ch_str
feature_name13 = 'trend_change_' + current_ch_str
biolinq_data[feature_name12] = biolinq_data[feature_name10].subtract(biolinq_data[feature_name11])
biolinq_data[feature_name13] = biolinq_data[feature_name12].diff().rolling(roll_win_size,min_periods=int(roll_win_size*roll_min_size_ratio)).median()
fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name12],name=feature_name12,legendgroup = '7'),row=3,col=2)
fig = fig.add_trace(go.Scatter(x=biolinq_data['Time'],y=biolinq_data[feature_name13],name=feature_name13,legendgroup = '8'),row=4,col=2)

In [ ]:
fig.show()